In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statistics import stdev
import numpy as np
from scipy import stats
import statsmodels
from statsmodels.sandbox.stats.multicomp import multipletests
from matplotlib import pyplot

import scipy.stats
import sklearn.linear_model
import sklearn.neighbors
from sklearn.naive_bayes import GaussianNB

import sklearn.ensemble
import time
import bp_preprocessing as preprocessing

In [31]:
dev = pd.read_csv("dev.csv")

In [33]:
dev = dev[['group',
           'AK1C1',
           'TAU',
           '1433G',
           'SCUB1',
           'FMOD',
           'AMYP',
           'CRIS3',
           'MYDGF',
           'RARR2',
           'ATS8',
           'PGK1',
           '1433Z',
           'SV2A',
           'TRH',
           'GUAD',
           'HV69D',
           'CO7',
           'SERC']]

In [34]:
# Create binary columns
dev['AD_MCI'] = np.where(dev['group'] == 'AD_MCI', 1, 0)
dev['Healthy'] = np.where(dev['group'] == 'Healthy', 1, 0)
dev['PD'] = np.where(dev['group'] == 'PD', 1, 0)
dev['PD_MCI_LBD'] = np.where(dev['group'] == 'PD_MCI_LBD', 1, 0)

In [35]:
dev

,group,AK1C1,TAU,1433G,SCUB1,FMOD,AMYP,CRIS3,MYDGF,RARR2,...,SV2A,TRH,GUAD,HV69D,CO7,SERC,AD_MCI,Healthy,PD,PD_MCI_LBD
0,AD_MCI,18.009949,18.421606,22.471329,19.499176,23.995281,20.762366,17.475345,21.729926,24.855334,...,20.768264,14.948166,22.269739,19.087294,24.369242,21.597414,1,0,0,0
1,Healthy,18.795230,18.255636,21.840747,19.720331,24.253167,20.503768,18.806484,21.872982,25.262400,...,20.442076,16.581877,21.154784,22.774641,24.419797,21.653803,0,1,0,0
2,Healthy,19.083557,19.129840,22.448146,19.344676,23.408567,20.256659,18.675593,21.908379,24.586243,...,20.530374,17.860609,21.420884,23.181192,24.466082,21.175356,0,1,0,0
3,AD_MCI,18.108128,19.934621,22.636775,19.684006,24.460110,20.351661,15.599738,21.678495,25.387522,...,20.327775,17.274123,21.917507,24.669925,24.514284,21.671200,1,0,0,0
4,Healthy,18.650715,18.557176,21.824173,19.585008,24.180328,20.276796,18.267226,21.830032,25.000770,...,19.883247,17.417208,20.485582,24.348318,24.870784,21.443932,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Healthy,19.123224,18.611889,21.705861,18.992121,24.638736,20.232877,18.509092,21.895392,25.793417,...,20.821832,17.242076,20.781667,24.185051,25.093713,20.726307,0,1,0,0
234,Healthy,20.130518,18.380580,21.953330,19.903244,24.912696,20.371756,18.258177,21.607310,26.024950,...,20.122928,18.159375,20.124358,23.729045,24.579697,20.855775,0,1,0,0
235,Healthy,17.928082,19.516288,22.107162,20.307207,23.900876,20.081079,18.201544,21.600041,24.887781,...,20.803016,17.417667,21.175874,23.346395,24.761225,21.736080,0,1,0,0
236,Healthy,18.205782,18.081087,21.764129,19.689804,23.962881,20.814052,17.564997,22.016034,25.242071,...,19.948511,16.731749,20.427924,25.587862,25.003600,21.338434,0,1,0,0


In [36]:
train, val = preprocessing.split_cats_by_tolerance(dev,tolerance=0.01,
                                                   silent=False,
                                                   randomstate=98281,
                                                   split=0.15,step=1,
                                                   categories=['Healthy','AD_MCI','PD','PD_MCI_LBD'])

{'Healthy': 112, 'AD_MCI': 37, 'PD_MCI_LBD': 27, 'PD': 26}
{'Healthy': 20, 'AD_MCI': 6, 'PD': 5, 'PD_MCI_LBD': 5}
Randstate: 98450

Percent Healthy in dev, test: [0.5544554455445545, 0.5555555555555556] 
Standard deviation of these values: 0.0005500550055005382 


Percent AD_MCI in dev, test: [0.18316831683168316, 0.16666666666666666] 
Standard deviation of these values: 0.008250825082508254 


Percent PD in dev, test: [0.12871287128712872, 0.1388888888888889] 
Standard deviation of these values: 0.0050880088008800894 


Percent PD_MCI_LBD in dev, test: [0.13366336633663367, 0.1388888888888889] 
Standard deviation of these values: 0.002612761276127612 



In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [40]:
def apply_ml_model_AD(dev, classifier):
    """
    finds the R2 score for different ML models
    """
    folds = sklearn.model_selection.LeaveOneOut().split(dev)

    fold_scores = []
    for train_indexes, val_indexes in folds: # the KFold splitter returns the indexes of the data, not the data itself
        train = dev.iloc[train_indexes]
        val = dev.iloc[val_indexes]

        scaler = sklearn.preprocessing.StandardScaler() # normalizing the numerical features
        train_X = scaler.fit_transform(train.iloc[:, 1:19])
        val_X = scaler.transform(val.iloc[:, 1:19])

        train_y = train.iloc[:,19]
        val_y = val.iloc[:,19]
        
        if classifier == "random_forest": 
            model = sklearn.ensemble.RandomForestClassifier()
        elif classifier == "naive_bayes":  
            model = sklearn.naive_bayes.GaussianNB()
        elif classifier == "decision_tree": 
            model = sklearn.tree.DecisionTreeClassifier()
        else: 
            print("wrong classifier named entered")
            
        model.fit(train_X, train_y) 
        fold_scores.append(
            sklearn.metrics.balanced_accuracy_score(val_y, model.predict(val_X)))
    return print(f"Leave-one-out cross validated balanced accuracy scores for {classifier} model (mean, std): ({np.mean(fold_scores)}, {np.std(fold_scores)})")

In [41]:
apply_ml_model_AD(dev, classifier = 'random_forest')

Leave-one-out cross validated balanced accuracy scores for random_forest model (mean, std): (0.8991596638655462, 0.30111719104485574)


In [42]:
dev = dev[dev['AD_MCI'] != 1]

In [43]:
def apply_ml_model_Healthy_vs_PD(dev, classifier):
    """
    finds the R2 score for different ML models
    """
    folds = sklearn.model_selection.LeaveOneOut().split(dev)

    fold_scores = []
    for train_indexes, val_indexes in folds: # the KFold splitter returns the indexes of the data, not the data itself
        train = dev.iloc[train_indexes]
        val = dev.iloc[val_indexes]

        scaler = sklearn.preprocessing.StandardScaler() # normalizing the numerical features
        train_X = scaler.fit_transform(train.iloc[:,1:19]) 
        val_X = scaler.transform(val.iloc[:,1:19])

        train_y = train.iloc[:,20]
        val_y = val.iloc[:,20]
        
        if classifier == "random_forest": 
            model = sklearn.ensemble.RandomForestClassifier()
        elif classifier == "naive_bayes":  
            model = sklearn.naive_bayes.GaussianNB()
        elif classifier == "decision_tree": 
            model = sklearn.tree.DecisionTreeClassifier()
        else: 
            print("wrong classifier named entered")
            
        model.fit(train_X, train_y) 
        fold_scores.append(
            sklearn.metrics.balanced_accuracy_score(val_y, model.predict(val_X)))
    return print(f"Leave-one-out cross validated balanced accuracy scores for {classifier} model (mean, std): ({np.mean(fold_scores)}, {np.std(fold_scores)})")

In [44]:
apply_ml_model_Healthy_vs_PD(dev, classifier = 'random_forest')

Leave-one-out cross validated balanced accuracy scores for random_forest model (mean, std): (0.7846153846153846, 0.4110888989503812)


In [13]:
dev = dev[dev['Healthy'] != 1]

In [47]:
def apply_ml_model_PD(dev, classifier):
    """
    finds the R2 score for different ML models
    """
    folds = sklearn.model_selection.LeaveOneOut().split(dev)

    fold_scores = []
    for train_indexes, val_indexes in folds: # the KFold splitter returns the indexes of the data, not the data itself
        train = dev.iloc[train_indexes]
        val = dev.iloc[val_indexes]

        scaler = sklearn.preprocessing.StandardScaler() # normalizing the numerical features
        train_X = scaler.fit_transform(train.iloc[:,1:19]) 
        val_X = scaler.transform(val.iloc[:,1:19])

        train_y = train.iloc[:,21]
        val_y = val.iloc[:,21]
        
        if classifier == "random_forest": 
            model = sklearn.ensemble.RandomForestClassifier()
        elif classifier == "naive_bayes":  
            model = sklearn.naive_bayes.GaussianNB()
        elif classifier == "decision_tree": 
            model = sklearn.tree.DecisionTreeClassifier()
        else: 
            print("wrong classifier named entered")
            
        model.fit(train_X, train_y)
        fold_scores.append(
            sklearn.metrics.balanced_accuracy_score(val_y, model.predict(val_X)))
    return print(f"Leave-one-out cross validated balanced accuracy scores for {classifier} model (mean, std): ({np.mean(fold_scores)}, {np.std(fold_scores)})")

In [48]:
apply_ml_model_PD(dev, classifier = 'decision_tree')

Leave-one-out cross validated balanced accuracy scores for decision_tree model (mean, std): (0.7435897435897436, 0.43665093245965136)
